In [145]:
import pandas as pd
import numpy as np
from typing import Optional
from datetime import date, datetime
import math
import pickle
import numpy
from dataclasses import asdict, dataclass

In [74]:
# Удалить Дата исполнения заказа/договора* Месяц Год Дата поставки SWAP Эрикссон
expected_april = pd.read_excel('4.Ожидаемые поставки_Апрель2021.xlsx', header=3, sheet_name=0) #.drop(['Дата исполнения заказа/договора*', 'Месяц', 'Год', 'Дата поставки SWAP Эрикссон'], axis=1)
expected_april_set = set(expected_april.columns)

In [75]:
# Удалить длина-высота-ширина
expected_december = pd.read_excel('4.Ожидаемые поставки_Декабрь2020.xlsx', header=0, sheet_name=1) #.drop(['Длина', 'Ширина', 'Высота'], axis=1)
expected_december_set = set(expected_december.columns)

In [76]:
expected_january = pd.read_excel('4.Ожидаемые поставки_Январь2021.xlsx', header=0, sheet_name=1, skipfooter=7) #.drop(['Дата исполнения заказа/договора', 'Месяц', 'Год', 'Длина', 'Ширина', 'Высота'], axis=1)
expected_january_set = set(expected_january.columns)

In [86]:
intersected = expected_april_set & expected_december_set & expected_january_set
expected_april = expected_april[intersected]
expected_december = expected_december[intersected]
expected_january = expected_january[intersected]

In [89]:
concated_df = pd.concat([expected_april, expected_december, expected_january], axis=0, ignore_index=True)
with open('concated_df.pkl', 'wb') as f:
    pickle.dump(concated_df, f)

In [77]:
with open('concated_df.pkl', 'rb') as f:
    concated_df=pickle.load(f)
concated_df.head()
# len(concated_df)
# MSK: 101, К04, 107, К05
# SZ: 304
# Ural: 702

,Организация-получатель,Дата исполнения заказа/договора,Номер строки в заказе/релизе,В пути,Общий объём,Код позиции ЕНС,Наименование позиции ЕНС,Единица измерения,Полученное количество,Объём,Заказанное количество
0,101,13.05.2021,1,34,1.7,160.346.000041,"Блок внутренний IDU Huawei RTN 910AF, 2хGE RJ4...",штука,0,"0,05",34
1,101,13.05.2021,2,136,0.060928,249.022.000011,"Приемопередатчик оптический (eSFP, 1310нм, 1.2...",штука,0,0.000448,136
2,101,13.05.2021,3,8,0.16,161.346.000136,"Радиоблок внешний ODU Huawei RTN XMC-5D 18ГГц,...",штука,0,0.02,8
3,101,13.05.2021,4,8,0.16,161.346.000137,"Радиоблок внешний ODU Huawei RTN XMC-5D 18ГГц,...",штука,0,0.02,8
4,101,13.05.2021,5,9,0.18,161.346.000132,"Радиоблок внешний ODU Huawei RTN XMC-5D 18ГГц,...",штука,0,0.02,9


In [74]:
def add_noice(df):
    for idx, row in df.iterrows():
        df.at[idx,'Общий объём'] += numpy.random.randint(0, 50)

In [198]:
concated_df=concated_df.dropna(axis=0)
for idx, row in concated_df.iterrows():
    if isinstance(row['Общий объём'], float):
        continue
    concated_df.at[idx,'Общий объём'] = float(row['Общий объём'].replace(',', '.'))
grouped = concated_df.groupby(['Организация-получатель', 'Дата исполнения заказа/договора'])['Общий объём'].sum().reset_index()
msk = grouped.loc[grouped['Организация-получатель'].isin(['101', 'К04', '107', 'К05'])].groupby('Дата исполнения заказа/договора')['Общий объём'].sum().reset_index()
northwest = grouped.loc[grouped['Организация-получатель'].isin(['304'])].groupby('Дата исполнения заказа/договора')['Общий объём'].sum().reset_index()
ural = grouped.loc[grouped['Организация-получатель'].isin(['702'])].groupby('Дата исполнения заказа/договора')['Общий объём'].sum().reset_index()

add_noice(msk), add_noice(ural), add_noice(northwest)
groupe_dfs = {'msk': msk, 'northwest': northwest, 'ural': ural}
groupe_dfs

{'msk':     Дата исполнения заказа/договора Общий объём
 0               2020-12-06 00:00:00   22.125169
 1                        01.02.2021   69.063135
 2                        01.02.2022  162.643654
 3                        01.03.2021     47.0229
 4                        01.03.2022   31.470096
 ..                              ...         ...
 279                      31.08.2021   45.763344
 280                      31.10.2020     22.4008
 281                      31.12.2020      45.679
 282                      31.12.2021   83.551667
 283                      31.12.2022       34.27
 
 [284 rows x 2 columns],
 'northwest':     Дата исполнения заказа/договора Общий объём
 0               2021-01-22 00:00:00    15.63312
 1                        01.02.2021       6.185
 2                        01.02.2022   54.507779
 3                        01.03.2021  321.498313
 4                        01.04.2021  102.352702
 ..                              ...         ...
 196                  

In [146]:
expenditure_df = pd.read_excel('2.Факт расход в монтаж БС штуки 2021.xlsx', header=0, sheet_name=2, skipfooter=7) 
expenditure_df

,год,неделя,Расход,МРМосква,МРСеверозапад,МРУрал,Общий итог,Месяц
0,2017,40,БС,37.0,22.0,1.0,245,10
1,2017,40,модули/блоки,120.0,44.0,NaN,290,10
2,2017,41,БС,11.0,NaN,13.0,109,10
3,2017,41,модули/блоки,40.0,NaN,1.0,93,10
4,2017,42,БС,NaN,21.0,25.0,91,10
...,...,...,...,...,...,...,...,...
461,2022,14,модули/блоки,55.0,83.0,21.0,319,4
462,2022,15,БС,17.0,NaN,NaN,70,4
463,2022,15,модули/блоки,97.0,19.0,8.0,259,4
464,2022,16,БС,22.0,4.0,3.0,83,4


In [199]:
list_ = []
for idx, row in expenditure_df.iterrows():
    cur_date = datetime.strptime(f"{row['год']}-{row['неделя']}-1", "%Y-%W-%w")
    list_.append({'date': cur_date, 'msk': row['МРМосква'], 'northwest': row['МРСеверозапад'], 'ural': row['МРУрал']})
expand_df = pd.DataFrame(list_)
expand_df = expand_df.fillna(0.0).groupby('date').sum().reset_index().set_index('date')

for key, df in groupe_dfs.items():
    for idx, row in df.iterrows():
        result = expand_df.index.get_loc(row['Дата исполнения заказа/договора'], method='nearest')
        df.at[idx, 'Общий объём'] += expand_df.iloc[result][key]

groupe_dfs

{'msk':     Дата исполнения заказа/договора Общий объём
 0               2020-12-06 00:00:00  134.125169
 1                        01.02.2021  808.063135
 2                        01.02.2022  298.643654
 3                        01.03.2021    786.0229
 4                        01.03.2022  183.470096
 ..                              ...         ...
 279                      31.08.2021  379.763344
 280                      31.10.2020    104.4008
 281                      31.12.2020     249.679
 282                      31.12.2021  219.551667
 283                      31.12.2022      157.27
 
 [284 rows x 2 columns],
 'northwest':     Дата исполнения заказа/договора Общий объём
 0               2021-01-22 00:00:00    15.63312
 1                        01.02.2021     297.185
 2                        01.02.2022  108.507779
 3                        01.03.2021  612.498313
 4                        01.04.2021  393.352702
 ..                              ...         ...
 196                  

In [ ]:
with open('concated_df.pkl', 'rb') as f:
    concated_df=pickle.load(f)